https://colab.research.google.com/github/teddylee777/machine-learning/blob/master/04-TensorFlow2.0/01-%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-%EC%A3%BC%EA%B0%80%EC%98%88%EC%B8%A1/02-LSTM-stock-forecasting-with-LSTM-financedatareader.ipynb#scrollTo=qnaiLGuDbiJe

https://woochan-autobiography.tistory.com/871

In [39]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
data = pd.read_csv('삼성바이오로직스 주식 데이터.csv',encoding='cp949', parse_dates = ["날짜"])
data1 = pd.read_csv('삼성바이오로직스 주식 데이터(투자주체별).csv',encoding='cp949', parse_dates = ["날짜"])

In [41]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   종목코드    249 non-null    object        
 1   종목명     249 non-null    object        
 2   날짜      249 non-null    datetime64[ns]
 3   개인      249 non-null    int64         
 4   외국인     249 non-null    int64         
 5   기관계     249 non-null    int64         
 6   금융투자    249 non-null    int64         
 7   보험      249 non-null    int64         
 8   투신      249 non-null    int64         
 9   은행      249 non-null    int64         
 10  기타금융    249 non-null    int64         
 11  연기금     249 non-null    int64         
 12  기타법인    249 non-null    int64         
 13  기타외인    249 non-null    int64         
 14  사모펀드    249 non-null    int64         
dtypes: datetime64[ns](1), int64(12), object(2)
memory usage: 29.3+ KB


In [42]:
data.columns, data1.columns

(Index(['종목코드', '종목명', '날짜', '시간', '시가', '고가', '저가', '종가', '거래량', '거래대금',
        '누적체결매도수량', '누적체결매수수량'],
       dtype='object'),
 Index(['종목코드', '종목명', '날짜', '개인', '외국인', '기관계', '금융투자', '보험', '투신', '은행',
        '기타금융', '연기금', '기타법인', '기타외인', '사모펀드'],
       dtype='object'))

In [43]:
data.drop('종목코드', axis=1, inplace=True) 
data.drop('종목명', axis=1, inplace=True) 
data1.drop('종목코드', axis=1, inplace=True) 
data1.drop('종목명', axis=1, inplace=True) 

In [44]:
m_data = pd.merge(data, data1)
m_data.drop('누적체결매도수량', axis=1, inplace=True)
m_data.drop('누적체결매수수량', axis=1, inplace=True)

투자주체별 합치고 종가 삭제하고

In [45]:
# 날짜 - 날짜형으로 변경
m_data['날짜'] = pd.to_datetime(data['날짜'])

In [46]:
m_data['Year'] = m_data['날짜'].dt.year
m_data['Month'] = m_data['날짜'].dt.month
m_data['Day'] = m_data['날짜'].dt.day

#m_data.drop('날짜', axis = 1, inplace=True)
m_data['변화량'] = m_data['종가'] - m_data['시가']

In [47]:
m_data.columns

Index(['날짜', '시간', '시가', '고가', '저가', '종가', '거래량', '거래대금', '개인', '외국인', '기관계',
       '금융투자', '보험', '투신', '은행', '기타금융', '연기금', '기타법인', '기타외인', '사모펀드', 'Year',
       'Month', 'Day', '변화량'],
      dtype='object')

In [48]:

fi = m_data.drop(columns = ['시가', '고가', '저가', '종가', '거래량', '거래대금', '개인', '외국인', '기관계',
       '금융투자', '보험', '투신', '은행', '기타금융', '연기금', '기타법인', '기타외인', '사모펀드', ], inplace = False)

In [50]:
fi.to_csv('삼성바이오로직스_변화량.csv', encoding = 'utf-8')

In [ ]:
m_data.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# 스케일을 적용할 column을 정의합니다.
scale_cols = ['시간', '시가', '고가', '저가', '종가', '거래량', '거래대금',
       '개인', '외국인', '기관계', '금융투자', '보험', '투신', '은행', '기타금융', '연기금', '기타법인',
       '기타외인', '사모펀드', 'Year', 'Month', 'Day']
# 스케일 후 columns
scaled = scaler.fit_transform(m_data[scale_cols])
scaled

In [ ]:
df = pd.DataFrame(scaled, columns=scale_cols)

In [ ]:
df.info()

train/test 분리

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.drop('종가', 1), df['종가'], test_size=0.2, random_state=0, shuffle=False)

In [ ]:
x_test.columns

In [ ]:
x_train.info()

시퀀스 데이터셋 구성

In [ ]:
import tensorflow as tf

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
# 하이퍼 파라미터
WINDOW_SIZE=20
BATCH_SIZE=32

In [ ]:
# trian_data는 학습용 데이터셋, test_data는 검증용 데이터셋 입니다.
train_data = windowed_dataset(y_train, WINDOW_SIZE, BATCH_SIZE, True)
test_data = windowed_dataset(y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [ ]:
# 아래의 코드로 데이터셋의 구성을 확인해 볼 수 있습니다.
# X: (batch_size, window_size, feature)
# Y: (batch_size, feature)
for data in train_data.take(1):
    print(f'데이터셋(X) 구성(batch_size, window_size, feature갯수): {data[0].shape}')
    print(f'데이터셋(Y) 구성(batch_size, window_size, feature갯수): {data[1].shape}')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


model = Sequential([
    # 1차원 feature map 생성
    Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[WINDOW_SIZE, 1]),
    # LSTM
    LSTM(16, activation='tanh'),
    Dense(16, activation="relu"),
    Dense(1),
])

In [ ]:
# Sequence 학습에 비교적 좋은 퍼포먼스를 내는 Huber()를 사용합니다.
loss = Huber()
optimizer = Adam(0.0005)
model.compile(loss=Huber(), optimizer=optimizer, metrics=['mse'])

In [ ]:
import os

In [ ]:
# earlystopping은 10번 epoch통안 val_loss 개선이 없다면 학습을 멈춥니다.
earlystopping = EarlyStopping(monitor='val_loss', patience=10)
# val_loss 기준 체크포인터도 생성합니다.
filename = os.path.join('tmp', 'ckeckpointer.ckpt')
checkpoint = ModelCheckpoint(filename, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

In [ ]:
test_data 

In [ ]:
history = model.fit(train_data, 
                    validation_data=(test_data), 
                    epochs=50, 
                    callbacks=[checkpoint, earlystopping])

In [ ]:
model.load_weights(filename)

In [ ]:
print(test_data)

In [ ]:
pred = model.predict(test_data)

In [ ]:
pred

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(np.asarray(y_test)[20:], label='actual')
plt.plot(pred, label='prediction')
plt.legend()
plt.show()